# Save_Load_Models

<font color="blue">作者：欧新宇（Xinyu OU）</font>

<font color="red">本文档所展示的测试结果，均运行于：Intel Core i7-7700K CPU 4.2GHz</font>

保存和读取模型在实际应用中具有非常重要的意义，主要体现在以下两个方面：
1. 对于复杂的大型的模型，特别是那些需要花费很长时间来进行训练的模型，我们不可能每次都进行训练
2. 对于用于来说，它们的需求是应用模型，而不是训练模型

基于以上两个原因，我们希望训练和应用部署模型是分开的。sklearn提供了这种策略。

### 1. 载入数据及数据初始化

In [20]:
import sys
import os
import time
import numpy as np
# 导入多层感知机MLP神经网络
from sklearn.neural_network import MLPClassifier
from PIL import Image
import joblib
import load_MNIST

# TODO: 1: 载入数据集
start = time.time()

train_images = load_MNIST.load_train_images()
train_labels = load_MNIST.load_train_labels()
test_images = load_MNIST.load_test_images()
test_labels = load_MNIST.load_test_labels()

print("载入数据集共耗时: {:.3f}s".format(time.time() - start))


# TODO: 2. 数据预处理
# 标准调整形态的方法
# X_train = train_images.reshape(train_images.shape[0], train_images.shape[1]*train_images.shape[2])/255
# 此处，因为我们已经知道的样本的形态，所以可以直接书写值

X_train = train_images.reshape(60000, 28*28)/255
y_train = train_labels
X_test = test_images.reshape(10000, 28*28)/255
y_test = test_labels

# 为了提高训练速度，我们只提取10%的样本进行演示
X_train_lite = X_train[0:5999, :]
y_train_lite = y_train[0:5999]
X_test_lite = X_test[0:999, :]
y_test_lite = y_test[0:999]



 训练集图片大小: 28*28, 已载入60000/60000.
训练集标签数量: 60000...已完成。
 测试集图片大小: 28*28, 已载入10000/10000.
测试集标签数量: 10000...已完成。
载入数据集共耗时: 2.769s


### 2. 训练模型

In [21]:
# TODO: 3.训练MLP神经网络并输出预测结果
start = time.time()

print('开始训练模型，请稍等...', end='')

mlp = MLPClassifier(solver='lbfgs', hidden_layer_sizes=[
                    100, 100], activation='relu', alpha=1e-5, random_state=62)
mlp.fit(X_train_lite, y_train_lite)

print('训练结束，用时{:.2f}s.'.format(time.time() - start))

开始训练模型，请稍等...训练结束，用时5.26s.


### 3. 保存模型

In [22]:
# 保存mlp神经网络模型
ModelPath = os.path.join(os.getcwd(), 'tmp', 'Ch08MNIST_lbfgs.pkl')
joblib.dump(mlp, ModelPath)

print('训练集得分: {:.4f}, 测试集得分: {:.4f}'.format(
    mlp.score(X_train, y_train), mlp.score(X_test, y_test)))

训练集得分: 0.9355, 测试集得分: 0.9295


### 4. 载入模型

In [23]:
# TODO: 1: 载入模型
start = time.time()

ModelPath = os.path.join(os.getcwd(), 'tmp', 'Ch08MNIST_lbfgs.pkl')
model = joblib.load(ModelPath)

print("载入模型共耗时: {:.3f}s".format(time.time() - start))

载入模型共耗时: 0.001s


In [24]:
print(model)

MLPClassifier(activation='relu', alpha=1e-05, batch_size='auto', beta_1=0.9,
              beta_2=0.999, early_stopping=False, epsilon=1e-08,
              hidden_layer_sizes=[100, 100], learning_rate='constant',
              learning_rate_init=0.001, max_iter=200, momentum=0.9,
              n_iter_no_change=10, nesterovs_momentum=True, power_t=0.5,
              random_state=62, shuffle=True, solver='lbfgs', tol=0.0001,
              validation_fraction=0.1, verbose=False, warm_start=False)


### 5. 应用模型

In [25]:
# TODO: 2. 预测给定图像
# 此处我们属于自己的手写字体进行测试

filename = 'digital.png'
filepath = os.path.join(os.getcwd(), 'tmp', filename)
image = Image.open(filepath).convert('F')
image = image.resize((28, 28))


# 将PIL的Image图像转换为numpy的数组
img = np.asarray(image)
# 将二维矩阵转换为一维向量以便于预测
im = img.reshape(1, 28*28)

# 输出预测结果
pred = model.predict(im)[0]
print('预测结果是:{:.0f}'.format(pred))

# 显示图像
image.show()

预测结果是:7
